In [1]:
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, LSTM, LeakyReLU, Dropout
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import tensorflow as tf

In [2]:
data = pd.read_csv("simulation_Data_2020.csv")
data.head()

,Date_Time,t_in,t_out,temp_diff,power_heating,power_cooling,power_electricity,total_power
0,2020/01/01 01:00:00,11.60,-7.45,19.048198,7.31,0.0,0.2,7.31
1,2020/01/01 02:00:00,11.60,-7.56,19.160009,7.36,0.0,0.2,7.36
2,2020/01/01 03:00:00,11.58,-8.46,20.043659,7.63,0.0,0.2,7.63
3,2020/01/01 04:00:00,11.54,-9.18,20.710560,7.82,0.0,0.2,7.82
4,2020/01/01 05:00:00,11.49,-9.71,21.201178,7.97,0.0,0.2,7.97


In [19]:
power_electricity = data[['power_electricity']]
power_electricity_data = go.Scatter(x = data['Date_Time'], y = data['power_electricity'])
layout = go.Layout(title ='power_electricity time series', xaxis = dict(title='Date'), yaxis = dict(title='Power electricity'))

fig = go.Figure(data = [power_electricity_data], layout = layout)
fig.write_html('Scatter_Power_Electricity.html', auto_open = True)

In [20]:
min_max_scaler = MinMaxScaler()
norm_data = min_max_scaler.fit_transform(power_electricity.values)
norm_data
norm_data_df = pd.DataFrame(norm_data)
norm_data_df.head()

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [21]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []
    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size
    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, 1)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        labels.append(dataset[i+target_size])
    return np.array(data), np.array(labels)

In [22]:
past_history = 5
future_target = 0

TRAIN_SPLIT = int(len(norm_data) * 0.8)


x_train, y_train = univariate_data(norm_data,
                                   0,
                                   TRAIN_SPLIT,
                                   past_history,
                                   future_target)

x_test, y_test = univariate_data(norm_data,
                                 TRAIN_SPLIT,
                                 None,
                                 past_history,
                                 future_target)

In [23]:
num_units = 64
learning_rate = 0.0001
activation_function = 'sigmoid'
adam = Adam(lr=learning_rate)
loss_function = 'mse'
batch_size = 5
num_epochs = 50

# Initialize the RNN
model = Sequential()
model.add(LSTM(units = num_units, activation=activation_function, input_shape=(None, 1)))
model.add(LeakyReLU(alpha=0.5))
model.add(Dropout(0.1))
model.add(Dense(units = 1))

# Compiling the RNN
model.compile(optimizer=adam, loss=loss_function)

In [24]:
history = model.fit(
    x_train,
    y_train,
    validation_split=0.1,
    batch_size=batch_size,
    epochs=num_epochs,
    shuffle=False
)

Epoch 1/50
1261/1261 [==============================] - 11s 6ms/step - loss: 0.2587 - val_loss: 0.1653
Epoch 2/50
1261/1261 [==============================] - 6s 5ms/step - loss: 0.1765 - val_loss: 0.1153
Epoch 3/50
1261/1261 [==============================] - 7s 6ms/step - loss: 0.1277 - val_loss: 0.0802
Epoch 4/50
1261/1261 [==============================] - 8s 6ms/step - loss: 0.0999 - val_loss: 0.0645
Epoch 5/50
1261/1261 [==============================] - 7s 6ms/step - loss: 0.0856 - val_loss: 0.0593
Epoch 6/50
1261/1261 [==============================] - 7s 6ms/step - loss: 0.0779 - val_loss: 0.0568
Epoch 7/50
1261/1261 [==============================] - 7s 5ms/step - loss: 0.0760 - val_loss: 0.0550
Epoch 8/50
1261/1261 [==============================] - 7s 6ms/step - loss: 0.0735 - val_loss: 0.0537
Epoch 9/50
1261/1261 [==============================] - 7s 6ms/step - loss: 0.0699 - val_loss: 0.0522
Epoch 10/50
1261/1261 [==============================] - 8s 6ms/step - loss: 0.06

In [25]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))
epochs_list = [*range(0, 50, 1)]

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x = epochs_list, y = loss, mode = 'lines', name = 'Loss'))
fig2.add_trace(go.Scatter(x = epochs_list, y = val_loss, mode = 'lines', name = 'Loss Val'))
fig2.update_layout(
    title="Power electricity Loss VS Loss Val",
    xaxis_title="Epochs",
    yaxis_title="Loss and Loss Val",
#     legend_title="Legend Title",
)
# layout2 = go.Layout(title ='Power heating Loss VS Loss Val')
fig2.write_html('Power electricity Loss VS Los Val.html', auto_open = True)

In [26]:
original = pd.DataFrame(min_max_scaler.inverse_transform(y_test))
predictions = pd.DataFrame(min_max_scaler.inverse_transform(model.predict(x_test)))

In [27]:
predictions

,0
0,0.814244
1,0.814244
2,10.967048
3,10.530890
4,10.087976
...,...
1742,0.814244
1743,0.814244
1744,0.814244
1745,0.814244


In [28]:
original

,0
0,0.20
1,10.87
2,10.85
3,10.79
4,10.75
...,...
1742,0.20
1743,0.20
1744,0.20
1745,0.20


In [29]:
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x = original.index, y = original[0], mode = 'lines', name = 'Test Data'))
fig3.add_trace(go.Scatter(x = predictions.index, y = predictions[0], mode = 'lines', name = 'Predictions'))
fig3.update_layout(
    title="Original VS Predictions",
    xaxis_title="Days",
    yaxis_title="Original and Predictions",
#     legend_title="Legend Title",
)
# layout2 = go.Layout(title ='Power heating Loss VS Loss Val')
fig3.write_html('Original VS Predictions Power electricity.html', auto_open = True)